# - Sentimental analysis -
# @ 주제 : 한국인 메이저리거의 현지 평가 분석

# 1. 가설
* Naive Bayes 분류기를 사용하여 현재 한국인 메이저리거의 현지 평가가 긍정적인지 부정적인지 분석할 수 있다.

# 2. 연구방법
## 2.1 데이터 수집
* twitter에서 영문으로 작성된 메이저리거 관련 글을 수집하기로 함
* 한국인 메이저리거 관련 트윗의 수가 매우 적고 그 중 정서가 드러난 글은 더더욱 부족함
* 직접 다양한 해쉬 태그를 시도하여 메이저리거 관련 트윗 중 정서가 드러난 것들을 추림(오래 걸림..)
* 해당 document가 긍정적인 정서인 경우 1, 부정적인 정서면 0으로 class를 분류함

### - twitter API를 이용한 트위터 해쉬태그 검색

In [16]:
import twitter

# twitter API 사용을 위한 4가지 토큰
consumer_key="R8a6YSb4yf77X27emnGcI9hKdYYAEDbSqK08ocg6s5Vz9yLnr6"
consumer_secret="NIM6wPYiI2S4rCRkAPZUpaCpa"
token_key="VKTOHImjOvE4Kp8ECj3T11zzorHuwxUImNnfWGuuVIaK9"
token_secret="2373074766-PL14cizo2IrNPcvcUI25kdi7KjZQMAdRnNx2SPN"

api = twitter.Twitter(auth=twitter.OAuth(token_secret, token_key,consumer_secret, consumer_key))
    
response=api.search.tweets(q="#park")
str = response['statuses']
print(str)

[{u'contributors': None, u'truncated': False, u'text': u'Company profile THE PARK AT HILLSIDE, LLC - https://t.co/6PP9T0OeJw  #THE #PARK #AT #HILLSIDE, #LLC', u'is_quote_status': False, u'in_reply_to_status_id': None, u'id': 727322139460800512L, u'favorite_count': 0, u'entities': {u'symbols': [], u'user_mentions': [], u'hashtags': [{u'indices': [69, 73], u'text': u'THE'}, {u'indices': [74, 79], u'text': u'PARK'}, {u'indices': [80, 83], u'text': u'AT'}, {u'indices': [84, 93], u'text': u'HILLSIDE'}, {u'indices': [95, 99], u'text': u'LLC'}], u'urls': [{u'url': u'https://t.co/6PP9T0OeJw', u'indices': [44, 67], u'expanded_url': u'http://www.tennesseecompanies.org/the-park-at-hillside-llc-000340358', u'display_url': u'tennesseecompanies.org/the-park-at-hi\u2026'}]}, u'retweeted': False, u'coordinates': None, u'source': u'<a href="http://\xb4www.tennesseecompanies.org" rel="nofollow">Tennessee Companies</a>', u'in_reply_to_screen_name': None, u'in_reply_to_user_id': None, u'retweet_count': 0,

## 2.2 데이터 처리
* 수집한 document들을 단어 벡터로 변환하는 작업이 필요함
* 단어 벡터 중에서 관사나 이름 명사 등 정서와 관련없는 단어를 필터링하는 작업을 수행함
* 이 때, 해쉬태그를 포함한 단어도 정서를 나타내는 경우가 있어 이를 특수문자가 제거된 순수단어 벡터 형태로 변환하는 작업이 선행되어야 함



### - text parsing(특수 문자 및 길이가 1인 word 제거)

In [50]:
import numpy as np
import re

twit = []
DocList = []
regEx = re.compile('\\W*')

file =open('data/trainingDoc.txt')
for i in file.readlines():
    twit.append(i)

classVec = list(np.ones(105))
classVec.extend(np.zeros(105))

for i in range(len(twit)):
    listOfTokens = regEx.split(tw[i])
    DocList.append([tok.lower() for tok in listOfTokens if len(tok) > 2])

print DocList
print classVec

[['manny', 'machado', 'the', 'best', 'fantasy', 'baseball', 'player', 'trey', 'rose', 'aka', 'dynasty_digest', 'http', 'www', 'dobberbaseball', 'com', 'manny', 'machado', 'the', 'best', 'fantasy', 'baseball', 'player', 'trout', 'harper', 'machado'], ['machado', 'off', 'hot', 'start', 'you', 'think', 'allstar', 'mlb', 'orioles', 'http', '5nt5zf7pc'], ['manny', 'machado', 'freak', 'nature', 'dude', 'guy', 'just', 'makes', 'plays', 'that', 'one', 'else', 'would', 'dream', 'orioles', 'platinumglove', 'mannybeingmanny'], ['hey', 'birdland', 'let', 'just', 'use', 'this', 'excuse', 'bask', 'the', 'glory', 'that', 'machado', 'our', 'third', 'baseman'], ['machado', 'needs', 'signed', 'for', 'years', 'machado', 'orioles'], ['fantasy', 'recommendation', 'brett', 'talley', 'manny', 'machado', 'http', 'bit', '1sycmdh', 'alarm'], ['machado', 'just', 'damn', 'good'], ['that', 'moment', 'when', 'your', 'happy', 'your', 'recording', 'the', 'orioles', 'game', 'machado', 'fire'], ['back', 'back', 'homeru

### - 의미없는 단어를 필터 리스트에 등록하여 제거

In [51]:
filterList = ["seunghwanoh",'seung', 'hwan',"cardinals","stlouis","manny","machado", "orioles","mlb",
                "park","twins","maeda","kenta","dodgers","la","ladodgers","debut","hyunsookim",
                "kim","hyun","soo","baltimore","korean","tyler","colins","tigers",
                 "tylercollins","keuchel","houstonastros","giants","jepsen","braves",
                "cubs","redSox","greinke","puig","molina","detroitTigers","a","an","the",
                "for","by","and","of","or","to","is","are","has","have","about","i","you",
                "hyunjinryu", "hyunjin", "ryu", "la", "dodgers", "ladodgers",   
                "byeonghopark", "byeongho", "park", "twins", "minnesota", "minnesotatwins",
                "hyunsookim", "hyunsoo", "kim", "orioles", "baltimore", "baltimoreorioles",
                "daeholee", "daeho", "lee", "mariners", "seattle", "seattlemariners",
                "junghokang", "jungho", "kang", "pirates", "pittsburgh", "pittsburghpirtes",
                "sinsoochoo", "sinsoo", "choo", "rangers", "texas", "texasrangers",
                "darvish","osaka","yuDarvish","rangers","com"
                "he","they","http"," "]

for i in range(idx):
    temp = []
    for word in DocList[i]:
        word = word.upper().lower()
        if word not in filterList:
            temp.append(word)
   
    DocList[i] = temp

print DocList

[['best', 'fantasy', 'baseball', 'player', 'trey', 'rose', 'aka', 'dynasty_digest', 'www', 'dobberbaseball', 'com', 'best', 'fantasy', 'baseball', 'player', 'trout', 'harper'], ['off', 'hot', 'start', 'think', 'allstar', '5nt5zf7pc'], ['freak', 'nature', 'dude', 'guy', 'just', 'makes', 'plays', 'that', 'one', 'else', 'would', 'dream', 'platinumglove', 'mannybeingmanny'], ['hey', 'birdland', 'let', 'just', 'use', 'this', 'excuse', 'bask', 'glory', 'that', 'our', 'third', 'baseman'], ['needs', 'signed', 'years'], ['fantasy', 'recommendation', 'brett', 'talley', 'bit', '1sycmdh', 'alarm'], ['just', 'damn', 'good'], ['that', 'moment', 'when', 'your', 'happy', 'your', 'recording', 'game', 'fire'], ['back', 'back', 'homeruns', 'way', 'reimold', 'omazing', 'abc2news', 'ibackthebirds'], ['back', 'back', 'homers', 'let', 'homeruns', 'schoop'], ['homers', 'inning', 'reimold', 'scoop', 'mazing'], ['with', 'giving', 'boost', 'fantasy', 'ranks', 'draftkings', 'jimenez', 'with', 'those'], ['just', '

## 2.3 분류기 생성
* Naive Bayes 분류기를 이용함
* 232개의 document 중 210(90%)개를 traing set으로, 나머지 22(10%)개의 한국인 메이저리거 관련 document를 test set으로 사용
* 사전확률을 맞추기 위해 training set은 긍정 105개, 부정 105개. test set은 긍정 15개, 부정 7개로 구축.

In [47]:
import bayes
reload(bayes)

myVocabList = bayes.createVocabList(DocList)

trainMat=[]
int_classVec = []

for postinDoc in DocList:
    trainMat.append(bayes.setOfWords2Vec(myVocabList, postinDoc))

for str_class in classVec:
    int_class = int(str_class)
    int_classVec.append(int_class)
    
p0V,p1V,pAb=bayes.trainNB0(trainMat,int_classVec)


## 2.4 분석

In [71]:
file =open('data/testSet.txt')

testDoc=[]
regEx = re.compile('\\W*')

for i in file.readlines():
    testDoc.append(i)

classList = list(np.ones(15))
classList.extend(np.zeros(7))

errorCnt = 0
i = 0
for doc in testDoc:
    testEntry = regEx.split(doc)
    temp = np.array(bayes.setOfWords2Vec(myVocabList, testEntry))
    classified = bayes.classifyNB(temp,p0V,p1V,pAb)
    if classified != int(classList[i]):
        errorCnt += 1
    print 'class is ', classVec[i], 'classified as ', classified
    i += 1
    
print 'Total error rate : ', float(errorCnt)/len(testDoc)    

the word:  is not in my Vocabulary!
the word: Power is not in my Vocabulary!
the word: pitching is not in my Vocabulary!
the word: 94mph is not in my Vocabulary!
the word: strike is not in my Vocabulary!
the word: oh is not in my Vocabulary!
the word: seunghwanoh is not in my Vocabulary!
the word: cardinals is not in my Vocabulary!
the word: stlouis is not in my Vocabulary!
the word:  is not in my Vocabulary!
class is  1.0 classified as  0
the word:  is not in my Vocabulary!
the word: Cardinals is not in my Vocabulary!
the word: Korean is not in my Vocabulary!
the word: reliever is not in my Vocabulary!
the word: SeungHwanOh is not in my Vocabulary!
the word: http is not in my Vocabulary!
the word: koreatimesus is not in my Vocabulary!
the word: cardinals is not in my Vocabulary!
the word: oh is not in my Vocabulary!
the word: seung is not in my Vocabulary!
the word: hwan is not in my Vocabulary!
the word: in is not in my Vocabulary!
the word: relief is not in my Vocabulary!
the word: 

# 3. 결론
* 정확도 41% (에러율 : 59%)
* 한계
* 1) training data의 수가 부족하여 실제 test data에서는 긍정/부정을 판단불가능한 단어가 많이 발생함
* 2) 트위터 글의 특성상 정서를 나타내는 word가 1~2개에 불과하여 제대로 된 정서 분류가 어려움
* 3) 문법을 지키지 않은 단어 또는 오타가 분류 성능을 떨어뜨리는 요인이 됨
